# GeneriCorp Internal Tools

## Notebook Setup

In [1]:
# Importing the necessary Python libraries
import os
import json
import yaml
import pandas as pd
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [2]:
# Loading my personal API keys from file (not pushed to GitHub due to .gitignore file)
if os.path.exists('../../keys/api_keys.yaml'):
    with open('../../keys/api_keys.yaml') as f:
        api_keys = yaml.safe_load(f)

In [3]:
# Instantiating the chat model
try:
    chat_model = ChatOpenAI(api_key = os.environ['OPENAI_API_KEY'],
                            model = 'gpt-4o-mini')
except:
    chat_model = ChatOpenAI(api_key = api_keys['OPENAI_API_KEY'],
                            model = 'gpt-4o-mini')

In [4]:
# Loading the company history from file
with open('../../genericorp/about-gc/company-history.md', 'r', encoding = 'utf-8') as file:
    company_history = file.read()

In [5]:
# Loading the list of GeneriCorp tools and descriptions from file
with open('genericorp_internal_tools.json', 'r') as file:
    genericorp_internal_tools = json.load(file)

## Generating Knowledge Item Topic Ideas

In [6]:
# Creating the topic generation prompt
TOPIC_GENERATION_PROMPT = '''Imagine a fictional company called GeneriCorp. Their company history is as follows:

{company_history}

Please generate a list of potential knowledge items that might support the internal tool named "{tool_name}". Here is a description of the tool:

{tool_description} 

Consider the tool's purpose, features, and how it integrates within GeneriCorp's operations.

Please output the knowledge items in a Python comma-separated list format. Do not provide any additional text aside from the Python list.
'''


# Instantiating the output parser
output_parser = CommaSeparatedListOutputParser()

# Setting up the chat prompt template
topic_generation_prompt_template = PromptTemplate(
    template = TOPIC_GENERATION_PROMPT + '\n{format_instructions}',
    input_variables = ['company_history', 'tool_name', 'tool_description'],
    partial_variables = {'format_instructions': output_parser.get_format_instructions()}
)

# Creating the topic generation chain
topic_generation_chain = topic_generation_prompt_template | chat_model | output_parser

In [7]:
# Setting up the cache directory
TOPICS_CACHE_DIR = 'knowledge_topics_cache'
os.makedirs(TOPICS_CACHE_DIR, exist_ok = True)

# Dictionary to store all knowledge topics
all_knowledge_topics = {}

# Iterating through each tool
for tool_name, tool_description in genericorp_internal_tools.items():

    # Creating a cache filename for this specific tool
    cache_file = f"{TOPICS_CACHE_DIR}/{tool_name.replace(' ', '_').lower()}_topics.json"
    
    # Checking if cached topics exist for this tool
    if os.path.exists(cache_file):

        # Loading topics from cache
        with open(cache_file, 'r') as f:
            tool_topics = json.load(f)
    else:
        # Generating topics using the AI model for this tool
        tool_topics = topic_generation_chain.invoke({
            "company_history": company_history,
            "tool_name": tool_name,
            "tool_description": tool_description
        })
        
        # Caching the results
        with open(cache_file, 'w') as f:
            json.dump(tool_topics, f)
    
    # Store topics for this tool
    all_knowledge_topics[tool_name] = tool_topics
    
    # Print results for this tool
    print(f"\nTool: {tool_name}")
    print(f"Number of topics: {len(tool_topics)}")
    print("Topics:", tool_topics)


Tool: GenOps Suite
Number of topics: 15
Topics: ['`HR management best practices', 'finance workflow automation guidelines', 'procurement process optimization', 'logistics and supply chain tracking techniques', 'internal ticketing system usage', 'standard operating procedures for GenOps Suite', 'troubleshooting common issues in GenOps', 'integration methods with existing systems', 'data management and reporting techniques', 'user training resources for employees', 'case studies on workflow automation success', 'compliance and regulatory requirements', 'cost-saving strategies in operations', 'performance metrics for enterprise operations', 'FAQs about the GenOps Suite`']

Tool: CircuitForge
Number of topics: 15
Topics: ['`user_manual', 'circuit_design_guidelines', 'simulation_test_cases', 'AI_diagnostics_integration', 'debugging_techniques', 'hardware_component_library', 'firmware_update_protocols', 'troubleshooting_guide', 'performance_metrics_analysis', 'circuit_schematic_examples', '

## Generating the Knowledge Items from the Topics

In [8]:
# Creating the knowledge item generation prompt
KNOWLEDGE_ITEM_PROMPT_TEMPLATE = '''Imagine a fictional company called GeneriCorp. Their company history is as follows: 

{company_history}

Pretend you’re a college student writing a short, casual essay about the topic "{topic}" as it relates to the GeneriCorp tool: "{tool_name}". Below is a description about the tool:

{tool_description}

The end goal is that your short essay will become a knowledge item for GeneriCorp employees to reference. Keep the tone friendly and down-to-earth, but ensure the content remains coherent and informative—like you’re explaining it to classmates who missed a lecture.

Explain what this topic is about, why it matters for GeneriCorp, and how it connects to the tool’s purpose or features. Blend in any relevant details from GeneriCorp’s history or background when it naturally fits. Wrap up with a brief reflection on why this knowledge item might be valuable for the company moving forward.

Only provide the article text in your final response—no bullet points, disclaimers, or special formatting. Keep it relaxed, but organized enough to be a useful reference for GeneriCorp.'''

# Setting up the chat prompt template
knowledge_item_prompt_template = PromptTemplate(
    template = KNOWLEDGE_ITEM_PROMPT_TEMPLATE,
    input_variables = ['company_history', 'topic', 'tool_name', 'tool_description'],
)

# Creating the knowledge item generation chain
knowledge_item_generation_chain = knowledge_item_prompt_template | chat_model

In [10]:
# Setting the directory for the internal tool pages
internal_tools_dir = '../../genericorp/internal-tools/'

# Iterating through each tool
for tool_name, tool_description in genericorp_internal_tools.items():

    # Setting the directory for the current tool
    current_tool_dir = f'{internal_tools_dir}/{tool_name}/'

    # Creating the tool directory if it doesn't exist
    os.makedirs(current_tool_dir, exist_ok = True)

    # Iterating over each proposed topic for the current tool
    for topic in all_knowledge_topics[tool_name]:

        # Creating the filename for the knowledge item
        filename = os.path.join(current_tool_dir, f"{topic.lower().replace(' ', '_')}.md")

        # Checking if the knowledge item already exists
        if os.path.exists(filename):
            continue

        # Generating the knowledge item using the AI model
        response = knowledge_item_generation_chain.invoke({
            "company_history": company_history,
            "topic": topic,
            "tool_name": tool_name,
            "tool_description": tool_description
        })

        # Saving the knowledge item to file
        with open(filename, 'w', encoding = 'utf-8') as file:
            file.write(response.content)

        print(f"Generated knowledge item for topic '{topic}' in tool '{tool_name}'")

Generated knowledge item for topic '`HR management best practices' in tool 'GenOps Suite'
Generated knowledge item for topic 'finance workflow automation guidelines' in tool 'GenOps Suite'
Generated knowledge item for topic 'procurement process optimization' in tool 'GenOps Suite'
Generated knowledge item for topic 'logistics and supply chain tracking techniques' in tool 'GenOps Suite'
Generated knowledge item for topic 'internal ticketing system usage' in tool 'GenOps Suite'
Generated knowledge item for topic 'standard operating procedures for GenOps Suite' in tool 'GenOps Suite'
Generated knowledge item for topic 'troubleshooting common issues in GenOps' in tool 'GenOps Suite'
Generated knowledge item for topic 'integration methods with existing systems' in tool 'GenOps Suite'
Generated knowledge item for topic 'data management and reporting techniques' in tool 'GenOps Suite'
Generated knowledge item for topic 'user training resources for employees' in tool 'GenOps Suite'
Generated k